# Dados

In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.animation as animation
import numpy as np
import math

In [3]:
breast=pd.read_csv('dados/breast.csv',header=None)
breast=breast.rename(columns={breast.columns[30]:'y'})
breast

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,y
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0.0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0.0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0.0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0.0


# Normalizador

In [4]:
class Nomalizer:
    def __init__(self):
        self.y_max=0.0
        self.y_min=0.0
        self.x_max=[]
        self.x_min=[]

    def fit(self,x,y):
        # Pega os valores minimos de 
        self.y_max=y['y'].max().tolist()
        self.y_min=y['y'].min().tolist()

        # Pega os valores maximos e minimos do X
        self.x_max=x.max().tolist()
        self.x_min=x.min().tolist()
    
    def normalize(self,x,y):
        y_norm=y.map(self.y_norm_aux)
        x_norm=x.copy()
        norm_index=0
        for column in x.columns:
            x_column=[]
            for i in range(len(x[column])):
                x_column.append((x[column][i]-self.x_min[norm_index])/(self.x_max[norm_index]-self.x_min[norm_index]))
                
            norm_index+=1
            x_norm[column]=x_column    
        
        return x_norm,y_norm
    
    def desnormalize(self,x_norm,y_norm):
        y=y_norm.map(self.y_desnorm_aux)
        x=x_norm.copy()
        norm_index=0
        for column in x_norm.columns:
            x_column=[]
            for i in range(len(x_norm[column])):
                x_column.append((x_norm[column][i]*(self.x_max[norm_index]-self.x_min[norm_index]))+self.x_min[norm_index])
                
            norm_index+=1
            x[column]=x_column    
        
        return x,y
    
    def y_norm_aux(self,y):
        return (y-self.y_min)/(self.y_max-self.y_min)
    
    def y_desnorm_aux(self,y):
        return (y*(self.y_max-self.y_min))+self.y_min

# modelo

In [ ]:
class LogisticClassifier:
  def __init__(self,opt='SGD'):
    self.w =[]
    self.losses=[]
    

  def fit(self, X, y, n, reg, lr=0.001, epochs=100):
    x_poli = self.polinomial_transfomation(X,n)# Transforma em polinomio
    df_bias = pd.DataFrame({'bias':[1]*len(x_poli)})
    X_bias=pd.concat([df_bias,x_poli], axis=1)
    self.w=np.ones(X_bias.shape[1])
    self.fit_SGD(X_bias,y,reg,lr,epochs)

  def polinomial_transfomation(self,x,n):
    df_columns = x.columns
    poli_columns={}
    self.n=n
    x_poli=x.copy()
    for column in df_columns:
      for i in range(2,n+1):
        poli_columns[f"{column}_{i}"] = x[column]**i
    poli_df = pd.DataFrame(poli_columns)
    x_poli = pd.concat([x,poli_df], axis=1)
    return x_poli


  def fit_SGD(self, X, y, reg, lr, epochs):
    for i in range(epochs):
      errors=[]
      indices = list(range(len(X)))
      random.shuffle(indices)

      for sample in indices:
        #Calculo dos erros
        y_pred=(X.iloc[sample]*self.w).sum()
        y_sig=1 / (1 + np.exp(-y_pred))
        error=y.iloc[sample]-y_sig
        errors.append(error)
        
        #ajuste dos Ws    
        
        w_arr = np.array(self.w)
        inter = lr*error.values*X.iloc[sample]
        w_novo = w_arr + inter
        self.w=w_novo


      #Calculo da loss
      #loss=self.loss_function(X, y, reg)
      #self.losses.append(loss)

  

  def loss_function(self, x, y,reg):
    w=np.array(self.w)
    x_arr=x.to_numpy()
    y_arr=y.to_numpy()

    N = len(y_arr)#Numero de amostras
    p = w.T @ x_arr#Produto interno para cada xi
    sigma = 1/(1+np.exp(-p))
    loss = -(1/N)*np.sum(y_arr*np.log(sigma)+(1-y_arr)*np.log(1-sigma)) #Calcula a perda e salva na lista
    return loss

  def predict(self, X):
    y_pred=[]# Lista para salvar as predições
    df_bias = pd.DataFrame({'bias':[1]*len(X)})
    x_poli = self.polinomial_transfomation(X,self.n)
    X_bias=pd.concat([df_bias,x_poli], axis=1)
    for sample in range(len(X_bias)):
      y_pred.append((X_bias.iloc[sample]*self.w).sum())# Calcula a predição e salva na lista
    return y_pred

In [6]:
x=breast.iloc[:, 0:30]  
y=breast[['y']]

In [7]:
norm=Nomalizer()
norm.fit(x,y)
norm_Trx,norm_Try=norm.normalize(x,y)

In [8]:
norm_Trx

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,0.336364,0.132056,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
565,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,0.349495,0.113100,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
566,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,0.267677,0.137321,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909
567,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,0.675253,0.425442,...,0.633582,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315


In [9]:
lc = LogisticClassifier()

In [10]:
lc.fit(norm_Trx,norm_Try,n=3, reg=0.01, lr=0.001, epochs=30)

In [11]:
lc.w

bias   -0.202027
0      -0.448767
1      -0.005337
2      -0.482757
3      -0.186274
          ...   
27_3   -0.372614
28_2    0.536384
28_3    0.734295
29_2    0.685691
29_3    0.844163
Name: 199, Length: 91, dtype: float64

In [41]:
import numpy as np
def atualizar_pesos(w, x_i, e_i, alpha):
    """
    Atualiza o vetor de pesos usando a regra w(t) = w(t-1) + α * e_i(t-1) * x_i.

    Parâmetros:
    w -- Vetor de pesos atual (numpy array ou lista)
    x_i -- Vetor de entrada correspondente (numpy array ou lista)
    e_i -- Erro calculado (float ou int)
    alpha -- Taxa de aprendizado (float)

    Retorna:
    w_novo -- Vetor de pesos atualizado (numpy array)
    """
    # Converte os vetores para numpy arrays, se necessário
    w = np.array(w)
    x_i = np.array(x_i)
    
    # Regra de atualização
    w_novo = w + alpha * e_i * x_i
    
    return w_novo
# Exemplo de pesos iniciais
w = [0.5, -0.5, 0.8]  # Pesos iniciais

# Entrada para a amostra i
x_i = [1, 2, 3]

# Erro calculado para a amostra i
e_i = 0.6

# Taxa de aprendizado
alpha = 0.01

# Atualiza os pesos
w_atualizado = atualizar_pesos(w, x_i, e_i, alpha)

print("Pesos atualizados:", w_atualizado)


Pesos atualizados: [ 0.506 -0.488  0.818]


In [11]:
np.ones((breast.shape[0], 1))

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],